In [24]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
from pythermalcomfort.models import pmv_ppd
from pythermalcomfort.utilities import v_relative, clo_dynamic
from pythermalcomfort.utilities import met_typical_tasks
from pythermalcomfort.utilities import clo_individual_garments

In [ ]:
tdb = 27  # dry bulb air temperature, [$^{\circ}$C]
tr = 25  # mean radiant temperature, [$^{\circ}$C]
v = 0.3  # average air speed, [m/s]
rh = 50  # relative humidity, [%]
activity = "Typing"  # participant's activity description
garments = ["Sweatpants", "T-shirt", "Shoes or sandals"]

met = met_typical_tasks[activity]  # activity met, [met]
icl = sum(
    [clo_individual_garments[item] for item in garments]
)  # calculate total clothing insulation

# calculate the relative air velocity
vr = v_relative(v=v, met=met)
# calculate the dynamic clothing insulation
clo = clo_dynamic(clo=icl, met=met)

# calculate PMV in accordance with the ASHRAE 55 2020
results = pmv_ppd(tdb=tdb, tr=tr, vr=vr, rh=rh, met=met, clo=clo, standard="ASHRAE")

In [25]:
T = 5 # demand response time (h)
N = 3 # the building consisting of N floors
Population = np.random.randint(30, 100, size=(N, T)) # population distribution in the building
DRdemand = 100 # DR demand during the response time (kWh)
to = np.random.randint(20, 30, size=(T,1)) # outdoor air temperature

In [26]:
def deltaTset_DR(deltaTset, population, to):
    """
    calculate the DRpotential matrix
    """
    DRpotential = np.zeros((N, T))
    for i in range(N):
        for j in range(T):
            DRpotential[i][j] = deltaTset[i][j] * (population[i][j]) * (to[j]+10)
    return DRpotential

# res = deltaTset_DR(np.full((N,T), -2), Population, np.random.randint(25,30, size = (T,1)))

In [28]:
from pyomo.environ import * 
modelA = ConcreteModel()

# declare dicision variables
modelA.deltaTset=Var([N,T],domain = Reals)
# modelA.deltaTset = Set(dimen = 2)
# declare objective function
modelA.comfortloss=Var(RangeSet(N-1,T-1),domain = Reals)

# declare constraints
def totalDRdemand(model):
    return deltaTset_DR(modelA.deltaTset, Population, to).sum() >= DRdemand
def deltaTset_range(model):
    return (-3, model.deltaTset[i][j], 3)

modelA.cons = ConstraintList()

for i in range(0, N):
    for j in range(1, T):
        modelA.cons.add(deltaTset_range(modelA))

modelA.cons.add(totalDRdemand(modelA))

def ComfortLoss(model):
    """
    calculate the comfort loss matrix
    """
    loss = np.zeros((N, T))
    for i in range(N):
        for j in range(T):
            loss[i][j] = model.deltaTset[i][j]
            loss[i][j] *= Population[i][j]
    return loss.sum()


modelA.obj = Objective(expr = ComfortLoss(modelA))

modelA.pprint()

KeyError: "Index '0' is not valid for indexed component 'deltaTset'"